In [1]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import librosa.display
import pandas as pd
import os, sys, re, pickle, glob
import os.path
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

In [2]:
audio_dataset_path='/Users/esaltuk/Desktop/ELEC491/archive' 
metadata=pd.read_csv('/Users/esaltuk/Desktop/ELEC491/archive/MLEndHWD_Audio_Attributes.csv')

In [3]:
metadata

,Public filename,Interpreter,Song,Interpretation
0,0000.wav,216,Potter,Hum
1,0001.wav,100,Potter,Hum
2,0002.wav,177,Potter,Hum
3,0003.wav,159,Potter,Hum
4,0004.wav,160,Potter,Whistle
...,...,...,...,...
6606,6606.wav,204,Frozen,Hum
6607,6607.wav,94,Frozen,Hum
6608,6608.wav,164,Frozen,Hum
6609,6609.wav,125,Frozen,Hum


def features_extractor(file):
    
    features=[]
    
    # if None, fs would be 22050
    audio_data, sample_rate = librosa.load(file,mono=True,sr=22050)
      

    stft = np.abs(librosa.stft(audio_data))
    
    mfcc = np.mean(librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40).T,axis=0)
    features.extend(mfcc) # 40 = 40

    
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    features.extend(chroma) # 12 = 52
    
    mel = np.mean(librosa.feature.melspectrogram(y=audio_data, sr=sample_rate).T,axis=0)
    features.extend(mel) # 128 = 180
    
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    features.extend(contrast) # 7 = 187

    
    return features

In [173]:
def features_extractor(file):
    features=[]    
    audio_data, sample_rate = librosa.load(file,mono=True,sr=2*22050)
    
    mfcc = np.mean(librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40).T,axis=0)
    features.extend(mfcc) # 40 = 40
    
    return features

In [243]:
extracted_features=[]

for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),"MLEndHWD_"+str(row["Song"])+"_Audio_Files"+'/',str(row["Public filename"]))
    final_class_labels=row["Song"]
    Interpret = row["Interpretation"]
    if os.path.exists(file_name) == True:
      data=features_extractor(file_name)
      extracted_features.append([data,Interpret,final_class_labels])

6611it [03:03, 36.09it/s]


In [244]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','Interp','class'])

In [245]:
extracted_features_df

,feature,Interp,class
0,"[-371.79715, 127.446526, 2.6394265, 3.1592183,...",Hum,Potter
1,"[-505.87384, 130.31326, 22.047081, 20.901613, ...",Hum,Potter
2,"[-541.1099, 134.42548, 16.201548, 21.419962, 2...",Hum,Potter
3,"[-470.17938, 146.83046, 3.9286509, 8.309087, -...",Hum,Potter
4,"[-388.46826, 84.75418, 17.812887, -8.884424, 4...",Whistle,Potter
...,...,...,...
6606,"[-476.7946, 124.91712, 4.0885925, 38.76391, 26...",Hum,Frozen
6607,"[-489.08978, 121.73104, 46.9286, 34.83716, 30....",Hum,Frozen
6608,"[-402.44797, 130.2624, 16.161236, 15.045093, 4...",Hum,Frozen
6609,"[-476.2789, 157.25711, 44.029285, 17.612562, -...",Hum,Frozen


In [246]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.utils import to_categorical

le = LabelEncoder()

int_lbls=le.fit_transform(np.array(extracted_features_df.loc[:,"Interp"]))
class_lbls=le.fit_transform(np.array(extracted_features_df.loc[:,"class"]))

Determine whether the input is Humming or Whistle

In [248]:
X=np.array(pd.Series.tolist(extracted_features_df.feature)) # all the features

y=int_lbls # hum or whistle

In [249]:
X_train, X_rem, y_train, y_rem = train_test_split(X,y,test_size=0.3)

In [250]:
X_val, X_test, y_val, y_test = train_test_split(X_rem,y_rem,test_size=0.1)

In [251]:
print('Size of training set:', X_train.shape, y_train.shape)
print('Size of validation set:', X_val.shape, y_val.shape)
print('Size of testing set:', X_test.shape, y_test.shape)
     

Size of training set: (4627, 40) (4627,)
Size of validation set: (1785, 40) (1785,)
Size of testing set: (199, 40) (199,)


In [252]:
model  = svm.SVC(C=1)
model.fit(np.array(X_train),y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))

Training Accuracy 0.9814134428355306
Validation  Accuracy 0.9725490196078431


In [253]:
extracted_features_df["Interp"]=int_lbls
extracted_features_df["class"]=class_lbls

Splitting the Hums and Whistles for later use!

In [254]:
hums=pd.DataFrame(extracted_features_df[extracted_features_df["Interp"]==0])

In [255]:
whistles=pd.DataFrame(extracted_features_df[extracted_features_df["Interp"]==1])

Training SVM Model for all classes

In [256]:
X=np.array(pd.Series.tolist(extracted_features_df.feature)) # all the features

y=class_lbls # classes

In [257]:
extracted_features_df['class'].unique()

array([4, 7, 3, 5, 1, 2, 6, 0])

In [258]:
X_train, X_rem, y_train, y_rem = train_test_split(X,y,test_size=0.3)

In [259]:
X_val, X_test, y_val, y_test = train_test_split(X_rem,y_rem,test_size=0.1)

In [260]:
print('Size of training set:', X_train.shape, y_train.shape)
print('Size of validation set:', X_val.shape, y_val.shape)
print('Size of testing set:', X_test.shape, y_test.shape)
     

Size of training set: (4627, 40) (4627,)
Size of validation set: (1785, 40) (1785,)
Size of testing set: (199, 40) (199,)


In [261]:
model  = svm.SVC(C=100, gamma=0.01, probability=True)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))
print('The support vectors are', model.support_vectors_.shape)

tcm = confusion_matrix(y_true=y_train, y_pred=yt_p)
tcm_n = tcm.astype('float') / tcm.sum(axis=1)[:, np.newaxis]

vcm = confusion_matrix(y_true=y_val, y_pred=yv_p)
vcm_n = vcm.astype('float') / vcm.sum(axis=1)[:, np.newaxis]

print('Training confusion matrix:\n {}\n'.format(tcm))
print('Validation confusion matrix:\n {}\n'.format(vcm))

print('Normalised training confusion matrix:\n {}\n'.format(tcm_n))
print('Normalised validation confusion matrix:\n {}\n'.format(vcm_n))

Training Accuracy 1.0
Validation  Accuracy 0.33501400560224087
The support vectors are (4627, 40)
Training confusion matrix:
 [[570   0   0   0   0   0   0   0]
 [  0 580   0   0   0   0   0   0]
 [  0   0 575   0   0   0   0   0]
 [  0   0   0 591   0   0   0   0]
 [  0   0   0   0 586   0   0   0]
 [  0   0   0   0   0 577   0   0]
 [  0   0   0   0   0   0 571   0]
 [  0   0   0   0   0   0   0 577]]

Validation confusion matrix:
 [[ 41   2   4   4 156   1   7   3]
 [  3  58   9   4 136   2   3   5]
 [  2  14  65   6 139   1  10   4]
 [  1   3   3  78 126   3   1   9]
 [  1   0   0  11 207   2   1   7]
 [  2   1   1  11 153  50   2   7]
 [  4   6  13   3 125   1  51   3]
 [  0   3   1  11 152   2   3  48]]

Normalised training confusion matrix:
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]

Normalised validation c

In [34]:
param_grid = {
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001],
    'kernel': ['rbf', 'poly', 'sigmoid']
    }

In [265]:
X_hums=hums["feature"] # all the features
y_hums=hums['class'] # hum or whistle

In [266]:
X_train_hums, X_rem, y_train_hums, y_rem = train_test_split(X,y,test_size=0.3)

In [267]:
X_val_hums, X_test_hums, y_val_hums, y_test_hums = train_test_split(X_rem,y_rem,test_size=0.1)

In [268]:
print('Size of training set:', X_train_hums.shape, y_train_hums.shape)
print('Size of validation set:', X_val_hums.shape, y_val_hums.shape)
print('Size of testing set:', X_test_hums.shape, y_test_hums.shape)

Size of training set: (4627, 40) (4627,)
Size of validation set: (1785, 40) (1785,)
Size of testing set: (199, 40) (199,)


In [269]:
model  = svm.SVC(C=10, gamma=0.1)
model.fit(X_train_hums,y_train_hums)

yt_p = model.predict(X_train_hums)
yv_p = model.predict(X_val_hums)

print('Training Accuracy of Humms', np.mean(yt_p==y_train_hums))
print('Validation  Accuracy Humms', np.mean(yv_p==y_val_hums))
print('The support vectors are', model.support_vectors_.shape)

Training Accuracy of Humms 0.9997838772422736
Validation  Accuracy Humms 0.11988795518207283
The support vectors are (4627, 40)


In [271]:
def testyourdata(filename):
    datatest=features_extractor(filename)
    datatest=np.reshape(datatest,(1,-1))
    test_result=model.predict(datatest)
    probabilities = model.predict_proba(datatest)
    print(np.sort(probabilities))
    print(list(le.inverse_transform(np.argpartition(probabilities, -3)[:,-3:][0])))
    return le.inverse_transform(np.reshape(test_result,(-1,1)))[0]

In [273]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from xgboost import XGBClassifier
import xgboost as xgb

xgbc = XGBClassifier()
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softmax', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1 , num_class=8)

xgbc.fit(X_train, y_train)

print('The training data metrics are:')
print('Accuracy Score:', accuracy_score(y_true=y_train, y_pred=xgbc.predict(X_train)))

print('The validation data metrics are:')
print('Accuracy Score:', accuracy_score(y_true=y_val, y_pred=xgbc.predict(X_val)))

print('\n')
print('The support vectors are', model.support_vectors_.shape, ('\n'))

tcm = confusion_matrix(y_true=y_train, y_pred=xgbc.predict(X_train))
tcm_n = tcm.astype('float') / tcm.sum(axis=1)[:, np.newaxis]

vcm = confusion_matrix(y_true=y_val, y_pred=xgbc.predict(X_val))
vcm_n = vcm.astype('float') / vcm.sum(axis=1)[:, np.newaxis]

print('Training confusion matrix:\n {}\n'.format(tcm))
print('Validation confusion matrix:\n {}\n'.format(vcm))

print('Normalised training confusion matrix:\n {}\n'.format(tcm_n))
print('Normalised validation confusion matrix:\n {}\n'.format(vcm_n))

The training data metrics are:
Accuracy Score: 1.0
The validation data metrics are:
Accuracy Score: 0.40672268907563025


The support vectors are (4627, 40) 

Training confusion matrix:
 [[570   0   0   0   0   0   0   0]
 [  0 580   0   0   0   0   0   0]
 [  0   0 575   0   0   0   0   0]
 [  0   0   0 591   0   0   0   0]
 [  0   0   0   0 586   0   0   0]
 [  0   0   0   0   0 577   0   0]
 [  0   0   0   0   0   0 571   0]
 [  0   0   0   0   0   0   0 577]]

Validation confusion matrix:
 [[105  15  20  13  20   7  21  17]
 [ 10  82  46  17   9  16  26  14]
 [ 20  34 108  18  16  14  23   8]
 [ 16  15  22  97  32  11  10  21]
 [ 15  12  16  34  96  22  19  15]
 [ 11   8  10  34  26 103  21  14]
 [ 18  13  48  16  19   9  72  11]
 [ 20  17  25  31  26  22  16  63]]

Normalised training confusion matrix:
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0

In [274]:
import tensorflow as tf
print(tf.__version__)
import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense, LSTM, Dropout,Activation,Flatten, Conv2D ,Conv1D, MaxPooling1D,MaxPooling2D, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from sklearn import metrics, model_selection
from tensorflow.keras import regularizers

2.12.0


In [275]:

### Training / Fitting / Optimizing Model #####
test_size = 0.3
epochs    = 100
batch_size= 10
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [276]:
classes= ['Frozen', 'Harry', 'Panther', 'StarWars', 'Rain','Hakuna', 'Mamma']
n_classes=len(classes)

In [277]:
X_train.shape

(4627, 40)

In [291]:
model = Sequential()

model.add(Input(shape=(40,)))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation('softmax'))
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [284]:
model = Sequential()
# Define the model
model.add(Input(shape=(40,)))

model.add(Dense(512, activation='relu'))

model.add(Dense(256, activation='relu'))

model.add(Dense(128, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [292]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 256)               10496     
                                                                 
 activation_5 (Activation)   (None, 256)               0         
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_24 (Dense)            (None, 256)               65792     
                                                                 
 activation_6 (Activation)   (None, 256)               0         
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_25 (Dense)            (None, 7)                

In [293]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_val, y_val),verbose=1)

Epoch 1/100
145/145 [==============================] - 1s 7ms/step - loss: 13.6626 - accuracy: 0.1232 - val_loss: 9.7749 - val_accuracy: 0.1462
Epoch 2/100
145/145 [==============================] - 1s 6ms/step - loss: 15.9777 - accuracy: 0.1366 - val_loss: 31.0667 - val_accuracy: 0.1232
Epoch 3/100
145/145 [==============================] - 1s 6ms/step - loss: 117.3036 - accuracy: 0.1210 - val_loss: 398.6978 - val_accuracy: 0.1221
Epoch 4/100
145/145 [==============================] - 1s 6ms/step - loss: 516.7823 - accuracy: 0.1277 - val_loss: 1719.6665 - val_accuracy: 0.1154
Epoch 5/100
145/145 [==============================] - 1s 6ms/step - loss: 1147.9557 - accuracy: 0.1178 - val_loss: 2351.7522 - val_accuracy: 0.1160
Epoch 6/100
145/145 [==============================] - 1s 6ms/step - loss: 2382.0310 - accuracy: 0.1264 - val_loss: 6554.8594 - val_accuracy: 0.1272
Epoch 7/100
145/145 [==============================] - 1s 6ms/step - loss: 3440.3501 - accuracy: 0.1202 - val_loss: 91

In [102]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],1))

In [170]:
# CNN Model

model = Sequential()

model.add(Conv1D(64, 3, activation='relu', input_shape=(186, 1)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# fit network
model.fit(X_train, y_train, epochs=100,validation_data=(X_val,y_val), verbose=1)

Epoch 1/100
145/145 [==============================] - 2s 10ms/step - loss: 587.5239 - accuracy: 0.1225 - val_loss: 4632.7500 - val_accuracy: 0.1339
Epoch 2/100
145/145 [==============================] - 1s 7ms/step - loss: 44351.1172 - accuracy: 0.1241 - val_loss: 54240.2578 - val_accuracy: 0.1120
Epoch 3/100
145/145 [==============================] - 1s 7ms/step - loss: 308890.5625 - accuracy: 0.1230 - val_loss: 330113.1875 - val_accuracy: 0.1199
Epoch 4/100
145/145 [==============================] - 1s 7ms/step - loss: 735493.7500 - accuracy: 0.1303 - val_loss: 658403.0625 - val_accuracy: 0.1120
Epoch 5/100
145/145 [==============================] - 1s 7ms/step - loss: 1493999.8750 - accuracy: 0.1273 - val_loss: 1568499.2500 - val_accuracy: 0.1473
Epoch 6/100
145/145 [==============================] - 1s 7ms/step - loss: 2498185.2500 - accuracy: 0.1197 - val_loss: 1670282.1250 - val_accuracy: 0.1199
Epoch 7/100
145/145 [==============================] - 1s 7ms/step - loss: 3722653.7

Epoch 53/100
145/145 [==============================] - 1s 7ms/step - loss: 593773312.0000 - accuracy: 0.1145 - val_loss: 468877376.0000 - val_accuracy: 0.1199
Epoch 54/100
145/145 [==============================] - 1s 7ms/step - loss: 502675360.0000 - accuracy: 0.1318 - val_loss: 300269504.0000 - val_accuracy: 0.1199
Epoch 55/100
145/145 [==============================] - 1s 7ms/step - loss: 584505984.0000 - accuracy: 0.1254 - val_loss: 720322944.0000 - val_accuracy: 0.1322
Epoch 56/100
145/145 [==============================] - 1s 7ms/step - loss: 659166144.0000 - accuracy: 0.1111 - val_loss: 295465952.0000 - val_accuracy: 0.1473
Epoch 57/100
145/145 [==============================] - 1s 7ms/step - loss: 566854400.0000 - accuracy: 0.1342 - val_loss: 479031040.0000 - val_accuracy: 0.1120
Epoch 58/100
145/145 [==============================] - 1s 7ms/step - loss: 693298560.0000 - accuracy: 0.1284 - val_loss: 584259648.0000 - val_accuracy: 0.1322
Epoch 59/100
145/145 [==================

In [85]:
model2 = Sequential()

# Add LSTM layer with 128 units and input shape of (190, 1)
model2.add(LSTM(units=192, input_shape=(186, 1), return_sequences=True))

# Add Dropout layer to reduce overfitting
model2.add(Dropout(0.2))

# Add another LSTM layer with 64 units
model2.add(LSTM(units=96, return_sequences=True))

# Add Dropout layer
model2.add(Dropout(0.2))

# Add a final LSTM layer with 32 units
model2.add(LSTM(units=48))

# Add Dropout layer
model2.add(Dropout(0.2))

# Add Dense layer with 8 output classes and softmax activation
model2.add(Dense(units=8, activation='softmax', kernel_regularizer=regularizers.l2(0.001)))

model2.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


# fit network
model2.fit(X_train, y_train, epochs=100,validation_data=(X_val,y_val), verbose=1)

Epoch 1/100
145/145 [==============================] - 8s 45ms/step - loss: 2.0831 - accuracy: 0.1591 - val_loss: 2.0658 - val_accuracy: 0.1630
Epoch 2/100
145/145 [==============================] - 6s 41ms/step - loss: 2.0605 - accuracy: 0.1651 - val_loss: 2.0739 - val_accuracy: 0.1574
Epoch 3/100
145/145 [==============================] - 6s 40ms/step - loss: 2.0545 - accuracy: 0.1764 - val_loss: 2.0683 - val_accuracy: 0.1462
Epoch 4/100
145/145 [==============================] - 6s 41ms/step - loss: 2.0516 - accuracy: 0.1787 - val_loss: 2.0624 - val_accuracy: 0.1585
Epoch 5/100
145/145 [==============================] - 6s 41ms/step - loss: 2.0437 - accuracy: 0.1848 - val_loss: 2.0512 - val_accuracy: 0.1681
Epoch 6/100
145/145 [==============================] - 6s 41ms/step - loss: 2.0408 - accuracy: 0.1800 - val_loss: 2.0560 - val_accuracy: 0.1636
Epoch 7/100
145/145 [==============================] - 6s 41ms/step - loss: 2.0382 - accuracy: 0.1839 - val_loss: 2.0454 - val_accuracy: